### Set up

In [1]:
import os

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

In [2]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [40]:
llm_story_emulator = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.8, max_tokens=200)
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.8)
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.8)

### Generate an example of student's writing

In [41]:
writing_assignment = "Write a story about a character who discovers a hidden world within their own mind, filled with their deepest desires and fears."

In [42]:
story = llm_story_emulator(writing_assignment)
print(story)



Alice was always a curious and imaginative person. As a child, she would often get lost in her own thoughts, creating fantastic worlds and adventures in her mind. As she grew older, Alice's vivid imagination became more subdued, and she found herself getting caught up in the stresses and routines of everyday life.

But one day, while taking a walk in the park, Alice stumbled upon a hidden door in a tree trunk. She couldn't resist her curiosity and pushed the door open, expecting to find a squirrel's nest or some other small creature's hiding spot. However, to her surprise, she found herself falling through a dark tunnel and landing in a new world.

As she looked around, Alice realized that she was in a realm unlike any she had ever seen before. The sky was a deep purple, and the trees were a vibrant shade of blue. The grass was a bright green, and the flowers were all different colors, some of which she had never seen before. Alice's heart was


### Construct the prompt and get feedback

In [45]:
prompt = f"""You are a helpful assistant for a student writing a story based on the following writing assignment: {writing_assignment}.
             Use concise expression to provide constructive pointers and suggestions to help the student improve their writing.
             Focus on creativity, specific details, structure, and clarity. Avoid writing full sentences.
             Student's writing: {story}
             """

print(llm(prompt))


- Consider starting the story with a more attention-grabbing sentence to hook the reader's interest.
- Use more descriptive language to paint a vivid picture of Alice's imaginative mind as a child.
- When describing the hidden door, use sensory details to make the reader feel as though they are also discovering it for the first time.
- Instead of saying "to her surprise," show Alice's reaction to falling through the dark tunnel and landing in the new world.
- Use more specific and unique descriptions of the new world that Alice discovers.
- Show how Alice's heart is feeling, rather than just stating it.
- Consider adding in what Alice's initial thoughts and emotions are when she first realizes she has entered a new world within her own mind.
- Use more active verbs and avoid passive phrasing, such as "Alice was always a curious and imaginative person". Instead, show her curiosity and imagination in action.
- Consider adding in more details about the stresses and routines of Alice's ev

In [64]:
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader(r"C:\Users\an.jiang\Projects\StorySmithNotebook\data")

documents = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(
#     # Set a really small chunk size, just to show.
#     chunk_size = 1000,
#     chunk_overlap  = 100,
# )

# docs = text_splitter.split_documents(documents)

In [70]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
print(f'Average length among {len(documents)} documents loaded is {avg_doc_length(documents)} characters.')

Average length among 1 documents loaded is 4970 characters.


In [12]:
writing_assignment = "Write a story about a character who discovers a hidden world within their own mind, filled with their deepest desires and fears."

In [13]:
story = llm(writing_assignment)
print(story)



Once upon a time, there lived a young man named John. He was always a curious, adventurous soul, always looking for something new to


In [14]:
system_template = (
        """You are a helpful assistant for a student writing an essay.
           Provide constructive pointers and suggestions to help the student improve their writing.
           Focus on specific details, structure, and clarity. Avoid writing full sentences."""
)
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [15]:
# system_template = (
#     """You are helping a student write a creative story based on the writing assignment between <assignment></assignment>.
#         Do not write the story for them, only provide ideas and suggestions.
        
#         <assignment>
#         {assignment}
#         </assignment>
        
#     """
# )
# system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# human_template = "{text}"
# human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [16]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# get a chat completion from the formatted messages
feedback = chat(chat_prompt.format_prompt(text=story).to_messages())

In [17]:
chat_prompt.format_prompt(assignment=writing_assignment, text=story).to_messages()

[SystemMessage(content='You are a helpful assistant for a student writing an essay.\n    Provide constructive pointers and suggestions to help the student improve their writing. \n    Focus on specific details, structure, and clarity. Avoid writing full sentences.        \n'),
 HumanMessage(content='\n\nOnce upon a time, there lived a young man named John. He was always a curious, adventurous soul, always looking for something new to')]

In [18]:
print(feedback.content)

explore. One day, as he was walking through the forest, he stumbled upon a hidden cave. Intrigued, he entered the cave, eager to uncover its secrets. Inside, he discovered a glowing crystal that emitted a mysterious aura. 

The crystal seemed to be calling out to him, urging him to touch it. John hesitated, unsure of what would happen if he did. But his curiosity got the better of him, and he reached out to touch the crystal. As soon as his fingers made contact, he felt a surge of energy flow through his body. 

Suddenly, John found himself transported to a different world. It was a land of enchantment and magic, unlike anything he had ever seen before. The sky was a vibrant shade of purple, and the plants and animals had a mystical glow to them. 

John soon realized that he had been granted incredible powers in this new world. He could control the elements and communicate with animals. With his newfound abilities, he set out on a quest to save the land from an evil sorcerer who was th